In [ ]:
import keras
print(keras.__version__)

In [1]:
#Importing Libraries
import sys
import os
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential

In [2]:
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
#Reading file
df=pd.read_csv(r'S:\ML Projects\Facial Expression\fer2013\fer2013.csv')

In [4]:
print(df["Usage"].value_counts)

<bound method IndexOpsMixin.value_counts of 0           Training
1           Training
2           Training
3           Training
4           Training
5           Training
6           Training
7           Training
8           Training
9           Training
10          Training
11          Training
12          Training
13          Training
14          Training
15          Training
16          Training
17          Training
18          Training
19          Training
20          Training
21          Training
22          Training
23          Training
24          Training
25          Training
26          Training
27          Training
28          Training
29          Training
            ...     
35857    PrivateTest
35858    PrivateTest
35859    PrivateTest
35860    PrivateTest
35861    PrivateTest
35862    PrivateTest
35863    PrivateTest
35864    PrivateTest
35865    PrivateTest
35866    PrivateTest
35867    PrivateTest
35868    PrivateTest
35869    PrivateTest
35870    PrivateTest
35871    Pr

In [5]:
#Defining variables
X_train,y_train,X_test,y_test=[],[],[],[]

In [6]:
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           y_test.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [7]:
print(f"X_train sample data:{X_train[0:2]}")
print(f"y_train sample data:{y_train[0:2]}")
print(f"X_test sample data:{X_test[0:2]}")
print(f"y_test sample data:{y_test[0:2]}")

X_train sample data:[array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32), array([151., 150., 147., ..., 193., 183., 184.], dtype=float32)]
y_train sample data:[0, 0]
X_test sample data:[array([254., 254., 254., ...,  42., 129., 180.], dtype=float32), array([156., 184., 198., ..., 172., 167., 161.], dtype=float32)]
y_test sample data:[0, 1]


In [8]:
X_train=np.array(X_train,'float32')
y_train=np.array(y_train,'float32')
X_test=np.array(X_test,'float32')
y_test=np.array(y_test,'float32')

In [9]:
#Normalizing the dataset between 0 and 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

In [10]:
#Reshaping the dataset so that keras accept it.
num_features=64
num_labels=7
batch_size=64
epochs=30
width,height=48,48

X_train=X_train.reshape(X_train.shape[0],width,height,1)
X_test=X_test.reshape(X_test.shape[0],width,height,1)

In [26]:
#Designing CNN in keras

from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization,Dense
#model = tensorflow.keras.Sequential()
model=Sequential()


In [19]:
#1st convolutional layer
model.add(Conv2D(num_features,kernel_size=(3,3),activation='relu',input_shape=(X_train.shape[1:])))
model.add(Conv2D(num_features,kernel_size=(3,3),activation='relu'))

In [20]:
#Taking maxium value from pool and using it as output
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
#Dropout so model does not overfit 
model.add(Dropout(0.5))

In [21]:
import keras_resnet
import keras_resnet.models

ModuleNotFoundError: No module named 'keras_resnet'

In [23]:
#2nd convolutional layer
model.add(Conv2D(num_features,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(num_features,kernel_size=(3,3),activation='relu'))


model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
 
model.add(Dropout(0.5))

In [ ]:
#3rd convolutional layer
model.add(Conv2D(2*num_features,(3,3),activation='relu'))
model.add(Conv2D(2*num_features,(3,3),activation='relu'))


model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())

#Adding Dense layers
model.add(Dense(2*2*num_features,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2*2*num_features,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels,activation='softmax'))

model.compile(loss=categorical_crossentropy,optimizers=Adam(),metrics=['accuracy'])

model.fit(X_train,y_train,batch_size=batch_size,
          epochs=epochs,
        verbose=1,
         validation_data=(X_test,y_test),
         shuffle=True)

Train on 28709 samples, validate on 3589 samples
Epoch 1/30
15424/28709 [===============>..............] - ETA: 9:27 - loss: 2268577386.7095 - accuracy: 0.0017

In [ ]:
#Saving the  model to  use it later on
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")